<a href="https://colab.research.google.com/github/Fairuza12/MSProject/blob/main/Dummy_code_Model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install easyocr transformers torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 kB 17.4 MB/s eta 0:00:00


In [ ]:
import easyocr
import pandas as pd
import os
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import torch

In [ ]:
text_reader = easyocr.Reader(['bn'])

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [ ]:
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-handwritten')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-base-handwritten')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/4.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Set device to GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=False)
              (key): Linear(in_features=768, out_features=768, bias=False)
              (value): Linear(in_features=768, out_features=768, bias=False)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_fea

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/MS_Project/Datasets/HCR_DATASET/VHD/Image_text_Mapping_1.xlsx')

In [ ]:
df = df.dropna()

In [ ]:
image_folder = '/content/drive/MyDrive/MS_Project/Datasets/HCR_DATASET/VHD/images'

In [ ]:
from PIL import Image

In [ ]:
# Function to fine-tune EasyOCR output with TrOCR
def refine_with_trocr(image_path):
    # Load and preprocess the image for TrOCR
    image = Image.open(image_path).convert("RGB")
    pixel_values = processor(image, return_tensors="pt").pixel_values.to(device)

    # Perform text prediction using TrOCR
    with torch.no_grad():
        generated_ids = model.generate(pixel_values)
        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return generated_text

In [ ]:
def process_image(num_images):
  predictions = []
  ground_truth = []

  # Loop through the first num_images rows of the dataframe
  for index,row in df.iloc[:num_images].iterrows():
    image_name = row['Image']
    true_text = row['Text']

    # Construct full image path
    image_path = os.path.join(image_folder,image_name)

    # Perform OCR on the image using EasyOCR
    easyocr_result = text_reader.readtext(image_path)
    easyocr_text = ' '.join([result[1] for result in easyocr_result])

    # Refine EasyOCR's result using TrOCR
    refined_text = refine_with_trocr(image_path)

    # Combine both EasyOCR and TrOCR outputs
    final_prediction = refined_text if len(refined_text) > len(easyocr_text) else easyocr_text


    # Print predictions for debugging
    print(f"EasyOCR: {easyocr_text} | TrOCR: {refined_text} | Final: {final_prediction} | True: {true_text}")

    # Append predicted and true text to their respective lists
    predictions.append(final_prediction)
    ground_truth.append(true_text)

  accuracy = calculate_accuracy(predictions, ground_truth)
  print(f"Accuracy for {num_images} images: {accuracy * 100:.2f}%")

In [ ]:
def calculate_accuracy(predicts,gr_truth):
  correct = 0
  total = len(predicts)
  for pred,truth in zip(predicts,gr_truth):
    # Clean up the text by removing extra spaces and newlines for comparison
    pred_cleaned = pred.strip().replace("\n", " ").replace("  ", " ")
    truth_cleaned = truth.strip().replace("\n", " ").replace("  ", " ")
    if pred_cleaned == truth_cleaned:
      correct += 1
  return correct/total

In [ ]:
process_image(50)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


EasyOCR: { ৬৩-৩ 911@ .4 @||-&৫ -?) '91৮-য)'৬19_-89|-0) '১@-&'|@-9} ১>৬৬ ৬%92 >%ঋ৬ ১&|93৩|@ @৩!- `2 ৩ @ 9|222 _৫& &৩ ৬9৬<@|@ ৬9|||@ ৬৩|&_ ১৯_৩৬৩ %৫, ৮৩৬|@ ঐ1৮ ৮@৫ 9৮৮ ১& @@ '&ৎ @%_ ৬@৮২৩_ 2@|_ ২9৫ @ঢ ৩_@_ ৩৮ ৩)|& 1) ৩|8||9]3)_ 9|_|2 0 ঈ@ @ ৩৬& ৬)_&|৬ '৩5)_ ১৫ @||4 @!@ & ৬১ 6 '৯৬_৫ ৩ 99, ৪ষ) 8|&@. | TrOCR: 1955 56 | Final: { ৬৩-৩ 911@ .4 @||-&৫ -?) '91৮-য)'৬19_-89|-0) '১@-&'|@-9} ১>৬৬ ৬%92 >%ঋ৬ ১&|93৩|@ @৩!- `2 ৩ @ 9|222 _৫& &৩ ৬9৬<@|@ ৬9|||@ ৬৩|&_ ১৯_৩৬৩ %৫, ৮৩৬|@ ঐ1৮ ৮@৫ 9৮৮ ১& @@ '&ৎ @%_ ৬@৮২৩_ 2@|_ ২9৫ @ঢ ৩_@_ ৩৮ ৩)|& 1) ৩|8||9]3)_ 9|_|2 0 ঈ@ @ ৩৬& ৬)_&|৬ '৩5)_ ১৫ @||4 @!@ & ৬১ 6 '৯৬_৫ ৩ 99, ৪ষ) 8|&@. | True: 20.txt
EasyOCR: হৃদয়' কথ্যল্রত্য' 'বনধো' ব্রতা 7প্পে ডাবস পাবঘ্লন   হতৈ সমা` 'কুদাডবো   প্রগঞন াঘার' কাবনে 'সাদ ধাতা; সবজ্ধনা" '~ন"ন' 'কতৈ' 5া%_ সতে   বিজ্ঞডাম ক্ষেপ্ত' তূ়' বাাহ গদ্দ 7ঘ @ে " ব্র হ্মান 'শান্ত' কন প-সষ্ ০~বাঁচচ ৮- দণ' 9় ?  'চাড়' , - 4 19 -মবং আদ > - সন' ন- ২-দুহ 7-ওদে | TrOCR: 1 References and their own | Final: হৃদয়' কথ্যল্রত্য' 'বনধো' ব্রতা 7প্পে ডাবস পাবঘ্ল

In [ ]:
process_image(100)

EasyOCR: { ৬৩-৩ 911@ .4 @||-&৫ -?) '91৮-য)'৬19_-89|-0) '১@-&'|@-9} ১>৬৬ ৬%92 >%ঋ৬ ১&|93৩|@ @৩!- `2 ৩ @ 9|222 _৫& &৩ ৬9৬<@|@ ৬9|||@ ৬৩|&_ ১৯_৩৬৩ %৫, ৮৩৬|@ ঐ1৮ ৮@৫ 9৮৮ ১& @@ '&ৎ @%_ ৬@৮২৩_ 2@|_ ২9৫ @ঢ ৩_@_ ৩৮ ৩)|& 1) ৩|8||9]3)_ 9|_|2 0 ঈ@ @ ৩৬& ৬)_&|৬ '৩5)_ ১৫ @||4 @!@ & ৬১ 6 '৯৬_৫ ৩ 99, ৪ষ) 8|&@. | TrOCR: 1955 56 | Final: { ৬৩-৩ 911@ .4 @||-&৫ -?) '91৮-য)'৬19_-89|-0) '১@-&'|@-9} ১>৬৬ ৬%92 >%ঋ৬ ১&|93৩|@ @৩!- `2 ৩ @ 9|222 _৫& &৩ ৬9৬<@|@ ৬9|||@ ৬৩|&_ ১৯_৩৬৩ %৫, ৮৩৬|@ ঐ1৮ ৮@৫ 9৮৮ ১& @@ '&ৎ @%_ ৬@৮২৩_ 2@|_ ২9৫ @ঢ ৩_@_ ৩৮ ৩)|& 1) ৩|8||9]3)_ 9|_|2 0 ঈ@ @ ৩৬& ৬)_&|৬ '৩5)_ ১৫ @||4 @!@ & ৬১ 6 '৯৬_৫ ৩ 99, ৪ষ) 8|&@. | True: 20.txt
EasyOCR: হৃদয়' কথ্যল্রত্য' 'বনধো' ব্রতা 7প্পে ডাবস পাবঘ্লন   হতৈ সমা` 'কুদাডবো   প্রগঞন াঘার' কাবনে 'সাদ ধাতা; সবজ্ধনা" '~ন"ন' 'কতৈ' 5া%_ সতে   বিজ্ঞডাম ক্ষেপ্ত' তূ়' বাাহ গদ্দ 7ঘ @ে " ব্র হ্মান 'শান্ত' কন প-সষ্ ০~বাঁচচ ৮- দণ' 9় ?  'চাড়' , - 4 19 -মবং আদ > - সন' ন- ২-দুহ 7-ওদে | TrOCR: 1 References and their own | Final: হৃদয়' কথ্যল্রত্য' 'বনধো' ব্রতা 7প্পে ডাবস পাবঘ্ল

In [ ]:
process_image(150)

EasyOCR: { ৬৩-৩ 911@ .4 @||-&৫ -?) '91৮-য)'৬19_-89|-0) '১@-&'|@-9} ১>৬৬ ৬%92 >%ঋ৬ ১&|93৩|@ @৩!- `2 ৩ @ 9|222 _৫& &৩ ৬9৬<@|@ ৬9|||@ ৬৩|&_ ১৯_৩৬৩ %৫, ৮৩৬|@ ঐ1৮ ৮@৫ 9৮৮ ১& @@ '&ৎ @%_ ৬@৮২৩_ 2@|_ ২9৫ @ঢ ৩_@_ ৩৮ ৩)|& 1) ৩|8||9]3)_ 9|_|2 0 ঈ@ @ ৩৬& ৬)_&|৬ '৩5)_ ১৫ @||4 @!@ & ৬১ 6 '৯৬_৫ ৩ 99, ৪ষ) 8|&@. | TrOCR: 1955 56 | Final: { ৬৩-৩ 911@ .4 @||-&৫ -?) '91৮-য)'৬19_-89|-0) '১@-&'|@-9} ১>৬৬ ৬%92 >%ঋ৬ ১&|93৩|@ @৩!- `2 ৩ @ 9|222 _৫& &৩ ৬9৬<@|@ ৬9|||@ ৬৩|&_ ১৯_৩৬৩ %৫, ৮৩৬|@ ঐ1৮ ৮@৫ 9৮৮ ১& @@ '&ৎ @%_ ৬@৮২৩_ 2@|_ ২9৫ @ঢ ৩_@_ ৩৮ ৩)|& 1) ৩|8||9]3)_ 9|_|2 0 ঈ@ @ ৩৬& ৬)_&|৬ '৩5)_ ১৫ @||4 @!@ & ৬১ 6 '৯৬_৫ ৩ 99, ৪ষ) 8|&@. | True: 20.txt
EasyOCR: হৃদয়' কথ্যল্রত্য' 'বনধো' ব্রতা 7প্পে ডাবস পাবঘ্লন   হতৈ সমা` 'কুদাডবো   প্রগঞন াঘার' কাবনে 'সাদ ধাতা; সবজ্ধনা" '~ন"ন' 'কতৈ' 5া%_ সতে   বিজ্ঞডাম ক্ষেপ্ত' তূ়' বাাহ গদ্দ 7ঘ @ে " ব্র হ্মান 'শান্ত' কন প-সষ্ ০~বাঁচচ ৮- দণ' 9় ?  'চাড়' , - 4 19 -মবং আদ > - সন' ন- ২-দুহ 7-ওদে | TrOCR: 1 References and their own | Final: হৃদয়' কথ্যল্রত্য' 'বনধো' ব্রতা 7প্পে ডাবস পাবঘ্ল